In [1]:

import os
import math
import time
from datetime import datetime
import logging
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import yaml
import torch
from torch import nn
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import datasource, causal_cnn_models, modules, net_utils

In [2]:
logger = logging.getLogger(__name__)

def log(msg):
    logger.debug(msg)


def config_logger(log_file=None):
    r"""Config logger."""
    global logger
    logger.handlers.clear()
    logger.setLevel(logging.DEBUG)
    
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # create file handler which logs even debug messages
    if log_file:
        fh = logging.FileHandler(log_file)
        fh.setFormatter(format)
        logger.addHandler(fh)


def viz_epoch_batch(epoch, x_batch, x_hat_batch, log_path):
    # folder = os.path.join("logs", "recon_vae", log_filename)
    folder = os.path.join(log_path, "recon_vae")
    if not os.path.exists(folder):
        os.makedirs(folder)
    x_batch = x_batch.detach().cpu().numpy()
    x_hat_batch = x_hat_batch.detach().cpu().numpy()
    for i in range(1):
        orig = x_batch[i, 0, :]
        recon = x_hat_batch[i, 0, :]
        _, ax = plt.subplots()
        ax.plot(range(len(orig)), orig)
        # plt.savefig(
        #     f"{folder}/epoch{epoch}_item{i}_orig.png",
        #     format='png', dpi=300, bbox_inches='tight')
        ax.plot(range(len(recon)), recon)
        # plt.ylim((0, 2)) 
        plt.savefig(
            f"{folder}/epoch{epoch}_item{i}.png",
            format='png', dpi=300, bbox_inches='tight')
    plt.close()

    
def save_models(model_file_instance_pairs):
    for model_file in model_file_instance_pairs.keys():
        net = model_file_instance_pairs.get(model_file)
        torch.save(net.state_dict(), model_file)

def load_models(model_file_instance_pairs, device="cpu"):
    for model_file in model_file_instance_pairs.keys():
        net = model_file_instance_pairs.get(model_file)
        net.load_state_dict(
            torch.load(model_file, map_location=device))
    # No return seems necessary, in-memory models updated.

net_utils.fix_randomness()

In [3]:
r"Validation"
def init_load_models(params):
    """prepare model"""
    params_decoder = params['decoder'].copy()
    params_decoder['width'] = params['hz'] * params['seg_len_sec']
    net = causal_cnn_models.FoldVaeClassifFoldWeight(
        params['encoder'], params_decoder, n_split=params['n_split'], 
        n_class=params['n_class'], log=log, debug=False,
    )

    model_files = [
        f"{params['model_path']}/fold0_net.pt", 
    ]
    model_instances = [net]
    load_models({
        model_files[0]: model_instances[0].to(DEVICE),
    }, device=DEVICE)
    net.eval()
    return model_instances


In [4]:
r"Validation"
def init_load_models_z(params):
    """prepare model"""
    params_decoder = params['decoder'].copy()
    params_decoder['width'] = params['hz'] * params['seg_len_sec']
    net = causal_cnn_models.FoldVaeClassifFoldWeightZ(
        params['encoder'], params_decoder, n_split=params['n_split'], 
        n_class=params['n_class'], log=log, debug=False,
    )

    model_files = [
        f"{params['model_path']}/fold0_net.pt", 
    ]
    model_instances = [net]
    load_models({
        model_files[0]: model_instances[0].to(DEVICE),
    }, device=DEVICE)
    net.eval()
    return model_instances


In [6]:
test_rec_names = ['mesa-sleep-1790', 'mesa-sleep-1803', 'mesa-sleep-1901', 'mesa-sleep-1917']
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
ds = datasource.MesaDbCsv(
    f"{os.path.expanduser('~')}/data/mesa/polysomnography", data_subdir="set1x20",
    hz=100, class_map=class_map, n_subjects=-1, hz_rr=5, 
    filter_records=test_rec_names,
    is_rr_sig=False, is_rsp=False, is_ecg_beats=False,
)

Data base-dir:/home/habib/data/mesa/polysomnography, data:set1x20, hz:100, class_map:{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 2},
No label for annot '9' in mesa-sleep-1901_annot.csv
[mesa-sleep-1901_annot.csv] n_seg:1439, clz_lbl_dist:{0: 665, 1: 670, 2: 104}
[mesa-sleep-1790_annot.csv] n_seg:1198, clz_lbl_dist:{0: 543, 1: 569, 2: 86}
[mesa-sleep-1803_annot.csv] n_seg:632, clz_lbl_dist:{0: 282, 1: 302, 2: 48}
[mesa-sleep-1917_annot.csv] n_seg:1435, clz_lbl_dist:{0: 661, 1: 651, 2: 123}
Total files:4, n_seg:4704, distribution:(array([0, 1, 2]), array([2151, 2192,  361]))


### FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923082611

In [7]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 10
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923082611/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models_z(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-23 15:31:02,444 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923082611/models', 'n_class': 3}
/tmp/ipykernel_271152/1596233174.py:

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 19200])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 300])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 300])
[[337 193  13]
 [ 75 433  61]
 [ 17  59  10]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[162   8 112]
 [113  98  91]
 [ 22   7  19]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[409 131 125]
 [  0 661   9]
 [  0 102   2]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[363 272  26]
 [  2 541 108]
 [  0  76  47]]


### FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922190750

In [6]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 1
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922190750/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models_z(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-23 08:33:01,783 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 1, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922190750/models', 'n_class': 3}
/tmp/ipykernel_271152/1596233174.py:60

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 192000])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 3000])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 3000])
[[402 130  11]
 [177 351  41]
 [ 31  53   2]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[206   0  76]
 [ 64   3 235]
 [ 18   2  28]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[463 190  12]
 [289 372   9]
 [ 43  61   0]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[423 230   8]
 [ 25 610  16]
 [  2 110  11]]


### FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922213406

In [5]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922213406/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models_z(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-23 08:30:04,484 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 5, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/FoldVaeClassifFoldWeightZ_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922213406/models', 'n_class': 3}
/tmp/ipykernel_271152/1596233174.py:60

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 38400])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 600])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 600])
[[336 166  41]
 [ 55 407 107]
 [  7  74   5]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[161   1 120]
 [ 46  21 235]
 [ 15   1  32]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[414 148 103]
 [ 34 549  87]
 [  3 100   1]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[322 272  67]
 [  2 585  64]
 [  0  85  38]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split15_ecgTrue_rrFalse_rspFalse_20240922084657

In [6]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 15
    params['encoder']['depth'] = 3
    params['decoder']['depth'] = 3
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split15_ecgTrue_rrFalse_rspFalse_20240922084657/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
print(networks)

2024-09-22 17:23:30,913 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 15, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 3, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 3, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split15_ecgTrue_rrFalse_rspFalse_20240922084657/models', 'n_class': 3}


[FoldVaeClassifFoldWeight(
  (encoder): CausalCNNVEncoder(
    (network): Sequential(
      (0): CausalCNN(
        (network): Sequential(
          (0): CausalConvolutionBlock(
            (causal): Sequential(
              (0): Conv1d(1, 128, kernel_size=(5,), stride=(1,), padding=(4,))
              (1): Chomp1d()
              (2): LeakyReLU(negative_slope=0.01)
              (3): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(4,))
              (4): Chomp1d()
              (5): LeakyReLU(negative_slope=0.01)
            )
            (upordownsample): Conv1d(1, 128, kernel_size=(1,), stride=(1,))
          )
          (1): CausalConvolutionBlock(
            (causal): Sequential(
              (0): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(8,), dilation=(2,))
              (1): Chomp1d()
              (2): LeakyReLU(negative_slope=0.01)
              (3): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(8,), dilation=(2,))
              (4): Chomp

/tmp/ipykernel_3880218/1596233174.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(model_file, map_location=device))


In [11]:
for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        # input_rr = batch_data['rr']
        label = batch_data['label'].detach().cpu().numpy()[0]
        
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

label distribution: ['0:543', '1:569', '2:86']
1198 1198
[[334 167  42]
 [138 362  69]
 [ 19  64   3]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[122   1 159]
 [ 14  52 236]
 [  9   2  37]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[410  87 168]
 [  0 416 254]
 [  0  62  42]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[356 255  50]
 [ 11 575  65]
 [  0 117   6]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352

In [14]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-22 17:55:51,683 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 5, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352/models', 'n_class': 3}
/tmp/ipykernel_262861/1596233174.py:60: Fu

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 38400])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 600])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 600])
[[303 143  97]
 [ 52 375 142]
 [ 10  64  12]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[132   0 150]
 [  5  16 281]
 [  7   0  41]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[410 153 102]
 [  7 602  61]
 [  1 103   0]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[360 254  47]
 [  4 573  74]
 [  0  77  46]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922095435

In [18]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 1
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922095435/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-22 18:08:47,027 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 2, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 1, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240922095435/models', 'n_class': 3}
/tmp/ipykernel_262861/1596233174.py:60: Fu

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 192000])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 3000])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 3000])
[[384 155   4]
 [151 405  13]
 [ 15  58  13]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[198   5  79]
 [ 58  51 193]
 [ 17   2  29]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[440 124 101]
 [  0 645  25]
 [  0  99   5]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[419 231  11]
 [ 26 589  36]
 [  1  85  37]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358

In [7]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 1
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-23 19:24:04,967 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 1, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358/models', 'n_class': 3}
/tmp/ipykernel_453062/1596233174.py:60: 

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 19200])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 300])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 300])
[[502  32   9]
 [391 160  18]
 [ 60  25   1]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[275   5   2]
 [182 111   9]
 [ 36   8   4]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[478 113  74]
 [119 506  45]
 [  7  94   3]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[461 193   7]
 [ 40 551  60]
 [  3  81  39]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923212500
w_folds sigmoid

In [7]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 10
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923212500/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-24 14:25:09,417 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 5, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923212500/models', 'n_class': 3}
/tmp/ipykernel_1892809/1596233174.py:60:

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 19200])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 300])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 300])
[[398 125  20]
 [190 322  57]
 [ 32  52   2]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[259   0  23]
 [ 43  38 221]
 [ 14   2  32]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[446 207  12]
 [ 87 582   1]
 [  6  98   0]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[385 264  12]
 [ 13 625  13]
 [  2 110  11]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653
w_fold sigmoid

In [8]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-24 14:40:49,692 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 5, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 5, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653/models', 'n_class': 3}
/tmp/ipykernel_1892809/1596233174.py:60: F

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 38400])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 600])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 600])
[[369 174   0]
 [121 446   2]
 [ 23  63   0]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[217   4  61]
 [ 25 125 152]
 [  9   9  30]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[493  95  77]
 [518 151   1]
 [ 74  29   1]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[424 230   7]
 [ 19 591  41]
 [  0 104  19]]


### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240923212948
w_fold sigmoid

In [9]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 1
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240923212948/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-24 14:47:06,572 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 5, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 1, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split1_ecgTrue_rrFalse_rspFalse_20240923212948/models', 'n_class': 3}
/tmp/ipykernel_1892809/1596233174.py:60: F

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 192000])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 3000])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 3000])
[[391 123  29]
 [195 325  49]
 [ 29  49   8]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[142   6 134]
 [ 28  53 221]
 [ 15   3  30]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[450 136  79]
 [110 548  12]
 [ 13  87   4]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[374 244  43]
 [ 12 603  36]
 [  0 107  16]]


## interpret

### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352

In [11]:
def viz(input_dict, output_dict, pred_label):
    ecg = input_dict['ecg']
    label = input_dict['label']
    w_folds = output_dict['w_folds']
    print(w_folds)

In [11]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        w_folds = net_outputs['w_folds']
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        viz(batch_data, net_outputs, pred_label)
        
        if i_batch > 0:
            break
    
    # df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    # PRED_PATH = params['model_path'].replace('models', 'preds')
    # if not os.path.exists(PRED_PATH):
    #         os.makedirs(PRED_PATH)
    # df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    # cm = confusion_matrix(df['label'], df['pred'])
    # print(cm)

2024-09-23 20:26:36,013 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 4, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 5, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352/models', 'n_class': 3}
/tmp/ipykernel_453062/1596233174.py:60: Fu

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 38400])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 600])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 600])
[[[3.7578816]
  [4.0928783]
  [4.1091123]
  [4.0690665]
  [4.0928807]]]
[[[3.8910453]
  [3.6651144]
  [3.7998817]
  [3.754382 ]
  [3.8289568]]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[[4.057015 ]
  [3.8933315]
  [3.9902282]
  [3.98851  ]
  [4.0107975]]]
[[[4.094333 ]
  [4.0213866]
  [3.9737496]
  [4.1505523]
  [4.1435766]]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[[3.8552692]
  [3.8565664]
  [3.9318542]
  [3.7820506]
  [3.8359659]]]
[[[3.8497415]
  [3.725282 ]
  [3.6938684]
  [3.7513123]
  [3.8710113]]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[[3.9553504]
  [3.9205894]
  [3.898716 ]
  [3.7342958]
  [3.5716352]]]
[[[3.9631627]
  [4.0528116]
  [3.8874521]
  [3.8552194]
 

### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358

In [12]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 10
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })

        viz(batch_data, net_outputs, pred_label)
        
        if i_batch > 0:
            break
        
    # df = pd.DataFrame.from_dict(outputs)

    # PRED_PATH = params['model_path'].replace('models', 'preds')
    # if not os.path.exists(PRED_PATH):
    #         os.makedirs(PRED_PATH)
    # df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    # cm = confusion_matrix(df['label'], df['pred'])
    # print(cm)

2024-09-23 20:36:05,325 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 4, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split10_ecgTrue_rrFalse_rspFalse_20240923171358/models', 'n_class': 3}
/tmp/ipykernel_453062/1596233174.py:60: 

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 19200])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 300])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 300])
[[[3.3500228]
  [2.9673443]
  [2.8897953]
  [2.8967133]
  [2.884591 ]
  [2.9173658]
  [2.8957458]
  [2.8873274]
  [2.902593 ]
  [2.884305 ]]]
[[[3.0590782]
  [3.211787 ]
  [3.1286564]
  [3.1204486]
  [3.2453623]
  [3.0135236]
  [3.2506585]
  [3.2497792]
  [3.1632965]
  [3.1217778]]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[[3.0755224]
  [3.0859282]
  [3.1682942]
  [2.8469477]
  [3.0505128]
  [3.2077549]
  [3.049256 ]
  [2.9630485]
  [3.0404098]
  [3.0135956]]]
[[[3.1640422]
  [3.0309644]
  [3.0764604]
  [3.0469217]
  [3.074613 ]
  [3.1716413]
  [2.9640095]
  [3.1125612]
  [2.9666576]
  [3.1040323]]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[[3.1071196]
  [3.1318352]
  [2.8964837]
  [3.1012

### simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653

In [12]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)



for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })

        viz(batch_data, net_outputs, pred_label)
        
        if i_batch > 0:
            break
        
    # df = pd.DataFrame.from_dict(outputs)

    # PRED_PATH = params['model_path'].replace('models', 'preds')
    # if not os.path.exists(PRED_PATH):
    #         os.makedirs(PRED_PATH)
    # df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    # cm = confusion_matrix(df['label'], df['pred'])
    # print(cm)

2024-09-24 15:13:59,282 - __main__ - DEBUG - {'batch_size': 128, 'cuda': 5, 'data_path': 'data/mesa/polysomnography/set1', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 200, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 5, 'age_classif': False, 'input_ecg': True, 'input_rr': False, 'input_rsp': False, 'input_ecg_beats': False, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240923212653/models', 'n_class': 3}
/tmp/ipykernel_1892809/1596233174.py:60: F

label distribution: ['0:543', '1:569', '2:86']
1198 1198
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 38400])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 600])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 600])
[[[0.96702933]
  [0.96865326]
  [0.96959376]
  [0.9681911 ]
  [0.9685289 ]]]
[[[0.9732972 ]
  [0.97599345]
  [0.97545195]
  [0.97697794]
  [0.9710222 ]]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[[0.9706645]
  [0.9730259]
  [0.9737205]
  [0.9729946]
  [0.9723219]]]
[[[0.9711194 ]
  [0.9730809 ]
  [0.97311586]
  [0.9737607 ]
  [0.9729188 ]]]
label distribution: ['0:665', '1:670', '2:104']
1439 1439
[[[0.9683159 ]
  [0.9712496 ]
  [0.9732074 ]
  [0.97241014]
  [0.97198063]]]
[[[0.9726098 ]
  [0.9708603 ]
  [0.97382385]
  [0.97243416]
  [0.971796  ]]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[[0.9749416 ]
  [0.97554636]
  [0.974323  ]
  [0.97070235]
  [0.966887  ]]]
[[[0.97568953]
  [0.9764133